# `Промышленное машинное обучение на Spark`
## `Занятие 04: Детали SQL и Spark`

### `Находнов Максим (nakhodnov17@gmail.com)`
#### `Москва, 2023`

О чём можно узнать из этого ноутбука:

* RDD API
* Pivot/Unpivot
* Window function
* UDF

Для эффективной работы UDF необходимо установить библиотеку, реализующую передачу данных между Spark и Python в Arrow формате:

In [1]:
! pip3 install pyspark pyarrow


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

conf = (
    SparkConf()
        .set('spark.ui.port', '4050')
        .setMaster('local[*]')
)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/05 20:47:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### `Spark RDD API`

* [Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
* [Документация](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)

In [3]:
data = [
    (1, 10),
    (2, 41),
    (0, 12),
    (2, 64),
    (2, 22),
    (1, 11),
    (0, 94),
]
dist_data = sc.parallelize(data)
dist_data

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
dist_data.collect()

[(1, 10), (2, 41), (0, 12), (2, 64), (2, 22), (1, 11), (0, 94)]

In [5]:
dist_data.map(lambda x: (x[0] + x[1], x[0])).collect()

[(11, 1), (43, 2), (12, 0), (66, 2), (24, 2), (12, 1), (94, 0)]

In [6]:
dist_df = dist_data.toDF(['key', 'value'])
dist_df, dist_df.collect()

(DataFrame[key: bigint, value: bigint],
 [Row(key=1, value=10),
  Row(key=2, value=41),
  Row(key=0, value=12),
  Row(key=2, value=64),
  Row(key=2, value=22),
  Row(key=1, value=11),
  Row(key=0, value=94)])

In [7]:
dist_data.groupByKey().mapValues(sum).take(2)

[(0, 106), (1, 21)]

In [8]:
! echo "Hello, sample RDD" > text.txt
! echo "This RDD contains three lines" >> text.txt
! echo "This is the last line" >> text.txt
! echo "" >> text.txt
! echo "Just kidding, it contains five lines" >> text.txt

In [9]:
text_data = sc.textFile('text.txt')
text_data, text_data.collect()

(text.txt MapPartitionsRDD[17] at textFile at DirectMethodHandleAccessor.java:104,
 ['Hello, sample RDD',
  'This RDD contains three lines',
  'This is the last line',
  '',
  'Just kidding, it contains five lines'])

In [10]:
distinct_words = (
    text_data
        .filter(lambda x: len(x))
        .flatMap(lambda x: x.split(' '))
        .distinct()
)
distinct_words.saveAsTextFile('words.txt')
print(distinct_words.toDebugString().decode())

(2) PythonRDD[25] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[21] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[20] at partitionBy at DirectMethodHandleAccessor.java:104 []
 +-(2) PairwiseRDD[19] at distinct at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/2223031120.py:5 []
    |  PythonRDD[18] at distinct at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/2223031120.py:5 []
    |  text.txt MapPartitionsRDD[17] at textFile at DirectMethodHandleAccessor.java:104 []
    |  text.txt HadoopRDD[16] at textFile at DirectMethodHandleAccessor.java:104 []


Для того, чтобы переиспользовать посчитанные значения в рамках текущей сессии стоит использовать метод `.cache`, который сохраняет результат вычислений в данной вершине в оперативной памяти.

Метод `.persist` позволяет сохранять промежуточные вычисления в рамках текущей сессии с более тонкой настройкой места хранения (жёсткий диск, оперативная память, ...)

In [11]:
distinct_words_cached = distinct_words.cache()
print(distinct_words_cached.toDebugString().decode())

(2) PythonRDD[25] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |  MapPartitionsRDD[21] at mapPartitions at PythonRDD.scala:145 [Memory Serialized 1x Replicated]
 |  ShuffledRDD[20] at partitionBy at DirectMethodHandleAccessor.java:104 [Memory Serialized 1x Replicated]
 +-(2) PairwiseRDD[19] at distinct at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/2223031120.py:5 [Memory Serialized 1x Replicated]
    |  PythonRDD[18] at distinct at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/2223031120.py:5 [Memory Serialized 1x Replicated]
    |  text.txt MapPartitionsRDD[17] at textFile at DirectMethodHandleAccessor.java:104 [Memory Serialized 1x Replicated]
    |  text.txt HadoopRDD[16] at textFile at DirectMethodHandleAccessor.java:104 [Memory Serialized 1x Replicated]


In [12]:
distinct_words_cached.collect()
print(distinct_words_cached.toDebugString().decode())

(2) PythonRDD[25] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |       CachedPartitions: 2; MemorySize: 298.0 B; DiskSize: 0.0 B
 |  MapPartitionsRDD[21] at mapPartitions at PythonRDD.scala:145 [Memory Serialized 1x Replicated]
 |  ShuffledRDD[20] at partitionBy at DirectMethodHandleAccessor.java:104 [Memory Serialized 1x Replicated]
 +-(2) PairwiseRDD[19] at distinct at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/2223031120.py:5 [Memory Serialized 1x Replicated]
    |  PythonRDD[18] at distinct at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/2223031120.py:5 [Memory Serialized 1x Replicated]
    |  text.txt MapPartitionsRDD[17] at textFile at DirectMethodHandleAccessor.java:104 [Memory Serialized 1x Replicated]
    |  text.txt HadoopRDD[16] at textFile at DirectMethodHandleAccessor.java:104 [Memory Serialized 1x Replicated]


Для со данных между сессиями сессиями можно использовать `.checkpoint`. Особенность этого метода — изменение графа вычислений. Цепочка вычислений для сохраняемого RDD будет удалена. Сокращение цепочки вычислений полезно в случае больших графов, например, в итеративных алгоритмах.

In [13]:
distinct_first_words = (
    text_data
        .filter(lambda x: len(x))
        .flatMap(lambda x: x.split(' ')[0])
        .distinct()
)

sc.setCheckpointDir('./checkpoints')

distinct_first_words.checkpoint()
print(distinct_first_words.toDebugString().decode())

(2) PythonRDD[30] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[29] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[28] at partitionBy at DirectMethodHandleAccessor.java:104 []
 +-(2) PairwiseRDD[27] at distinct at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/2415635507.py:5 []
    |  PythonRDD[26] at distinct at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/2415635507.py:5 []
    |  text.txt MapPartitionsRDD[17] at textFile at DirectMethodHandleAccessor.java:104 []
    |  text.txt HadoopRDD[16] at textFile at DirectMethodHandleAccessor.java:104 []


In [14]:
distinct_first_words.collect()
print(distinct_first_words.toDebugString().decode())

(2) PythonRDD[30] at RDD at PythonRDD.scala:53 []
 |  ReliableCheckpointRDD[31] at collect at /var/folders/3c/vr2463p11lz5fr_80mrg72gh0000gq/T/ipykernel_37355/656455338.py:1 []


### `Pivot/Unpivot`

In [15]:
! wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
! unzip -o ml-1m.zip

--2023-02-05 20:47:39--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Распознаётся files.grouplens.org (files.grouplens.org)… 128.101.65.152
Подключение к files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 5917549 (5,6M) [application/zip]
Сохранение в: «ml-1m.zip.1»

ml-1m.zip.1         100%[===================>]   5,64M  3,37MB/s    за 1,7s    

2023-02-05 20:47:41 (3,37 MB/s) - «ml-1m.zip.1» сохранён [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [16]:
schema = (
    T.StructType()
        .add('movie_id', T.IntegerType())
        .add('movie', T.StringType())
        .add('categories', T.StringType())
)
movies_df = (
    spark.read.format('csv')
      .option("header", False)
      .option("sep", '::')
      .schema(schema)
      .load('./ml-1m/movies.dat')
)

schema = (
    T.StructType()
        .add('user_id', T.IntegerType())
        .add('movie_id', T.IntegerType())
        .add('rating', T.FloatType())
        .add('timestamp', T.StringType())
)
ratings_df = (
    spark.read.format('csv')
      .option("header", False)
      .option("sep", '::')
      .schema(schema)
      .load('./ml-1m/ratings.dat')
)

schema = (
    T.StructType()
        .add('user_id', T.IntegerType())
        .add('gender', T.StringType())
        .add('age', T.IntegerType())
        .add('occupation', T.IntegerType())
        .add('zip-code', T.StringType())
)
users_df = (
    spark.read.format('csv')
      .option("header", False)
      .option("sep", '::')
      .schema(schema)
      .load('./ml-1m/users.dat')
)

+-------+--------+------+
|user_id|movie_id|rating|
+-------+--------+------+
|      1|    1193|   5.0|
|      1|    1193|   4.0|
|      1|     661|   3.0|
|      2|     661|   3.0|
|      3|    1193|   4.0|
|      3|    2355|   5.0|
+-------+--------+------+

groupBy(user_id).pivot(movie_id).agg(mean(rating))
groupBy(user_id).pivot(movie_id).agg(first(rating))

1 -> (1193, 5.0), (661, 3.0)
2 -> (661, 3.0)
3 -> (1193, 4.0), (2355, 5.0)

   1193 661 2355
1  4.5  3.0 None
2  None 3.0 None
3  4.0 None 5.0

In [17]:
ratings_df.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|      1|    1193|   5.0|978300760|
|      1|     661|   3.0|978302109|
|      1|     914|   3.0|978301968|
|      1|    3408|   4.0|978300275|
|      1|    2355|   5.0|978824291|
|      1|    1197|   3.0|978302268|
|      1|    1287|   5.0|978302039|
|      1|    2804|   5.0|978300719|
|      1|     594|   4.0|978302268|
|      1|     919|   4.0|978301368|
|      1|     595|   5.0|978824268|
|      1|     938|   4.0|978301752|
|      1|    2398|   4.0|978302281|
|      1|    2918|   4.0|978302124|
|      1|    1035|   5.0|978301753|
|      1|    2791|   4.0|978302188|
|      1|    2687|   3.0|978824268|
|      1|    2018|   4.0|978301777|
|      1|    3105|   5.0|978301713|
|      1|    2797|   4.0|978302039|
+-------+--------+------+---------+
only showing top 20 rows



In [18]:
pivot_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id')
        .agg(F.first(ratings_df.rating))
)
pivot_df.where(pivot_df.user_id == 148).toPandas()

23/02/05 20:47:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


23/02/05 20:47:50 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


,user_id,1,2,3,4,5,6,7,8,9,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,148,5.0,5.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
top_movies_df = ratings_df.groupBy(ratings_df.movie_id).agg(
    F.count(ratings_df.rating).alias('rates'),
    F.mean(ratings_df.rating).alias('avg_rating')
).sort('rates', ascending=False).limit(100)

top_movies_df.show()

+--------+-----+------------------+
|movie_id|rates|        avg_rating|
+--------+-----+------------------+
|    2858| 3428|4.3173862310385065|
|     260| 2991| 4.453694416583082|
|    1196| 2990| 4.292976588628763|
|    1210| 2883| 4.022892819979188|
|     480| 2672|3.7638473053892216|
|    2028| 2653| 4.337353938937053|
|     589| 2649| 4.058512646281616|
|    2571| 2590| 4.315830115830116|
|    1270| 2583|3.9903213317847466|
|     593| 2578|4.3518231186966645|
|    1580| 2538| 3.739952718676123|
|    1198| 2514| 4.477724741447892|
|     608| 2513| 4.254675686430561|
|    2762| 2459| 4.406262708418057|
|     110| 2443| 4.234957020057307|
|    2396| 2369| 4.127479949345715|
|    1197| 2318|4.3037100949094045|
|     527| 2304| 4.510416666666667|
|    1617| 2288| 4.219405594405594|
|    1265| 2278| 3.953028972783143|
+--------+-----+------------------+
only showing top 20 rows



In [20]:
top_movies = top_movies_df.rdd.map(lambda x: x.movie_id).collect()

In [21]:
pivot_top_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id', top_movies)
        .agg(F.first(ratings_df.rating))
)
pivot_top_df.where(pivot_df.user_id < 3).toPandas()

,user_id,2858,260,1196,1210,480,2028,589,2571,1270,...,2699,750,39,21,1393,2804,588,2406,1220,733
0,1,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN
1,2,4.0,NaN,5.0,4.0,5.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
pivot_top_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id', top_movies)
        .agg(F.first(ratings_df.rating))
        .fillna(3.0)
)
pivot_top_df.where(pivot_top_df.user_id < 3).toPandas()

,user_id,2858,260,1196,1210,480,2028,589,2571,1270,...,2699,750,39,21,1393,2804,588,2406,1220,733
0,1,3.0,4.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,...,3.0,3.0,3.0,3.0,3.0,5.0,4.0,3.0,3.0,3.0
1,2,4.0,3.0,5.0,4.0,5.0,4.0,4.0,4.0,3.0,...,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0


In [23]:
path = "./m5-forecasting-accuracy"

df_validation = (
    spark.read.format('csv')
      .option("inferSchema", True)
      .option("header", True)
      .option("sep", ',')
      .load(f"{path}/sales_train_validation.csv")
)
df_validation.limit(10).toPandas()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,1,0,0,0,0,0,0,2,0,2


In [24]:
unpivot_expression = "stack(2, 'd_1', d_1, 'd_2', d_2) as (d, sales)"
unpivot_df = (
    df_validation
        .select('id', F.expr(unpivot_expression))
)

unpivot_df.show(truncate=False)

+-----------------------------+---+-----+
|id                           |d  |sales|
+-----------------------------+---+-----+
|HOBBIES_1_001_CA_1_validation|d_1|0    |
|HOBBIES_1_001_CA_1_validation|d_2|0    |
|HOBBIES_1_002_CA_1_validation|d_1|0    |
|HOBBIES_1_002_CA_1_validation|d_2|0    |
|HOBBIES_1_003_CA_1_validation|d_1|0    |
|HOBBIES_1_003_CA_1_validation|d_2|0    |
|HOBBIES_1_004_CA_1_validation|d_1|0    |
|HOBBIES_1_004_CA_1_validation|d_2|0    |
|HOBBIES_1_005_CA_1_validation|d_1|0    |
|HOBBIES_1_005_CA_1_validation|d_2|0    |
|HOBBIES_1_006_CA_1_validation|d_1|0    |
|HOBBIES_1_006_CA_1_validation|d_2|0    |
|HOBBIES_1_007_CA_1_validation|d_1|0    |
|HOBBIES_1_007_CA_1_validation|d_2|0    |
|HOBBIES_1_008_CA_1_validation|d_1|12   |
|HOBBIES_1_008_CA_1_validation|d_2|15   |
|HOBBIES_1_009_CA_1_validation|d_1|2    |
|HOBBIES_1_009_CA_1_validation|d_2|0    |
|HOBBIES_1_010_CA_1_validation|d_1|0    |
|HOBBIES_1_010_CA_1_validation|d_2|0    |
+-----------------------------+---

In [25]:
(
    unpivot_df
        .groupBy(unpivot_df.id)
        .pivot('d')
        .agg(F.sum(unpivot_df.sales))
).show(truncate=False)

+-------------------------------+---+---+
|id                             |d_1|d_2|
+-------------------------------+---+---+
|FOODS_2_387_CA_1_validation    |0  |0  |
|HOBBIES_1_258_CA_1_validation  |0  |0  |
|FOODS_2_322_CA_1_validation    |0  |1  |
|FOODS_3_352_CA_1_validation    |0  |0  |
|FOODS_1_011_CA_1_validation    |2  |1  |
|HOBBIES_1_273_CA_1_validation  |1  |0  |
|HOBBIES_1_163_CA_1_validation  |0  |0  |
|FOODS_2_011_CA_1_validation    |1  |1  |
|FOODS_1_101_CA_2_validation    |0  |0  |
|HOBBIES_2_044_CA_2_validation  |0  |0  |
|FOODS_3_808_CA_1_validation    |22 |18 |
|HOUSEHOLD_1_179_CA_2_validation|9  |5  |
|HOBBIES_1_236_CA_2_validation  |0  |0  |
|HOBBIES_1_299_CA_2_validation  |0  |0  |
|FOODS_1_206_CA_2_validation    |3  |0  |
|FOODS_3_529_CA_1_validation    |0  |0  |
|FOODS_3_644_CA_1_validation    |3  |1  |
|FOODS_3_693_CA_1_validation    |0  |0  |
|FOODS_1_054_CA_2_validation    |4  |6  |
|HOUSEHOLD_1_157_CA_2_validation|0  |0  |
+-------------------------------+-

In [26]:
(
    df_validation
        .where(df_validation.id == 'FOODS_3_808_CA_1_validation')
        .select('id', 'd_1', 'd_2')
).show(truncate=False)

+---------------------------+---+---+
|id                         |d_1|d_2|
+---------------------------+---+---+
|FOODS_3_808_CA_1_validation|22 |18 |
+---------------------------+---+---+



### `Window function`

In [27]:
from pyspark.sql.window import Window

In [28]:
data = [
    ('James', 'Sales', 3000),
    ('Michael', 'Sales', 4600),
    ('Robert', 'Sales', 4100),
    ('Maria', 'Finance', 3000),
    ('Scott', 'Finance', 3300),
    ('Jen', 'Finance', 3900), 
    ('Jeff', 'Marketing', 3000),
    ('Kumar', 'Marketing', 2000),
    ('Saif', 'Sales', 4100)
]
 
columns= ["emp_id", "dept_id", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.printSchema(), df.show(truncate=False)

root
 |-- emp_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- salary: long (nullable = true)

+-------+---------+------+
|emp_id |dept_id  |salary|
+-------+---------+------+
|James  |Sales    |3000  |
|Michael|Sales    |4600  |
|Robert |Sales    |4100  |
|Maria  |Finance  |3000  |
|Scott  |Finance  |3300  |
|Jen    |Finance  |3900  |
|Jeff   |Marketing|3000  |
|Kumar  |Marketing|2000  |
|Saif   |Sales    |4100  |
+-------+---------+------+



(None, None)

In [29]:
avg_salaries_df = df.groupBy(df.dept_id).agg(F.mean(df.salary).alias('avg_salary'))
df.join(avg_salaries_df, on='dept_id', how='inner').show()

+---------+-------+------+----------+
|  dept_id| emp_id|salary|avg_salary|
+---------+-------+------+----------+
|    Sales|  James|  3000|    3950.0|
|    Sales|Michael|  4600|    3950.0|
|    Sales| Robert|  4100|    3950.0|
|  Finance|  Maria|  3000|    3400.0|
|  Finance|  Scott|  3300|    3400.0|
|  Finance|    Jen|  3900|    3400.0|
|Marketing|   Jeff|  3000|    2500.0|
|Marketing|  Kumar|  2000|    2500.0|
|    Sales|   Saif|  4100|    3950.0|
+---------+-------+------+----------+



In [30]:
wspec = Window.partitionBy('dept_id')
(
    df
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('avg_salary', F.mean(df.salary).over(wspec))
).show(truncate=False)

+-------+---------+------+------------------------+----------+
|emp_id |dept_id  |salary|salaries_list           |avg_salary|
+-------+---------+------+------------------------+----------+
|Maria  |Finance  |3000  |[3000, 3300, 3900]      |3400.0    |
|Scott  |Finance  |3300  |[3000, 3300, 3900]      |3400.0    |
|Jen    |Finance  |3900  |[3000, 3300, 3900]      |3400.0    |
|Jeff   |Marketing|3000  |[3000, 2000]            |2500.0    |
|Kumar  |Marketing|2000  |[3000, 2000]            |2500.0    |
|James  |Sales    |3000  |[3000, 4600, 4100, 4100]|3950.0    |
|Michael|Sales    |4600  |[3000, 4600, 4100, 4100]|3950.0    |
|Robert |Sales    |4100  |[3000, 4600, 4100, 4100]|3950.0    |
|Saif   |Sales    |4100  |[3000, 4600, 4100, 4100]|3950.0    |
+-------+---------+------+------------------------+----------+



In [31]:
wspec = Window.partitionBy('dept_id').orderBy('salary')
(
    df
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('row_number', F.row_number().over(wspec))
        .withColumn('avg_salary', F.mean(df.salary).over(wspec))
).show(truncate=True)

+-------+---------+------+--------------------+----------+------------------+
| emp_id|  dept_id|salary|       salaries_list|row_number|        avg_salary|
+-------+---------+------+--------------------+----------+------------------+
|  Maria|  Finance|  3000|              [3000]|         1|            3000.0|
|  Scott|  Finance|  3300|        [3000, 3300]|         2|            3150.0|
|    Jen|  Finance|  3900|  [3000, 3300, 3900]|         3|            3400.0|
|  Kumar|Marketing|  2000|              [2000]|         1|            2000.0|
|   Jeff|Marketing|  3000|        [2000, 3000]|         2|            2500.0|
|  James|    Sales|  3000|              [3000]|         1|            3000.0|
| Robert|    Sales|  4100|  [3000, 4100, 4100]|         2|3733.3333333333335|
|   Saif|    Sales|  4100|  [3000, 4100, 4100]|         3|3733.3333333333335|
|Michael|    Sales|  4600|[3000, 4100, 4100...|         4|            3950.0|
+-------+---------+------+--------------------+----------+------

In [32]:
wspec = (
    Window
        .partitionBy('dept_id')
        .orderBy('salary')
        .rowsBetween(Window.currentRow, Window.unboundedFollowing)
)
(
    df
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('avg_salary', F.mean(df.salary).over(wspec))
).show()

+-------+---------+------+--------------------+-----------------+
| emp_id|  dept_id|salary|       salaries_list|       avg_salary|
+-------+---------+------+--------------------+-----------------+
|  Maria|  Finance|  3000|  [3000, 3300, 3900]|           3400.0|
|  Scott|  Finance|  3300|        [3300, 3900]|           3600.0|
|    Jen|  Finance|  3900|              [3900]|           3900.0|
|  Kumar|Marketing|  2000|        [2000, 3000]|           2500.0|
|   Jeff|Marketing|  3000|              [3000]|           3000.0|
|  James|    Sales|  3000|[3000, 4100, 4100...|           3950.0|
| Robert|    Sales|  4100|  [4100, 4100, 4600]|4266.666666666667|
|   Saif|    Sales|  4100|        [4100, 4600]|           4350.0|
|Michael|    Sales|  4600|              [4600]|           4600.0|
+-------+---------+------+--------------------+-----------------+



In [33]:
wspec = Window.partitionBy('dept_id').orderBy('salary').rangeBetween(-400, 400)
(
    df
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('avg_salary', F.mean(df.salary).over(wspec))
).show()

+-------+---------+------+-------------+----------+
| emp_id|  dept_id|salary|salaries_list|avg_salary|
+-------+---------+------+-------------+----------+
|  Maria|  Finance|  3000| [3000, 3300]|    3150.0|
|  Scott|  Finance|  3300| [3000, 3300]|    3150.0|
|    Jen|  Finance|  3900|       [3900]|    3900.0|
|  Kumar|Marketing|  2000|       [2000]|    2000.0|
|   Jeff|Marketing|  3000|       [3000]|    3000.0|
|  James|    Sales|  3000|       [3000]|    3000.0|
| Robert|    Sales|  4100| [4100, 4100]|    4100.0|
|   Saif|    Sales|  4100| [4100, 4100]|    4100.0|
|Michael|    Sales|  4600|       [4600]|    4600.0|
+-------+---------+------+-------------+----------+



In [34]:
wspec = Window.partitionBy('dept_id').orderBy('salary')
(
    df
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('avg_salary', F.mean(df.salary).over(wspec))
        .withColumn('cume_dist', F.cume_dist().over(wspec))
        .withColumn('lag', F.lag(df.salary, 1).over(wspec))
        .withColumn('lead', F.lead(df.salary, 1).over(wspec))
).show(truncate=False) 

+-------+---------+------+------------------------+------------------+------------------+----+----+
|emp_id |dept_id  |salary|salaries_list           |avg_salary        |cume_dist         |lag |lead|
+-------+---------+------+------------------------+------------------+------------------+----+----+
|Maria  |Finance  |3000  |[3000]                  |3000.0            |0.3333333333333333|null|3300|
|Scott  |Finance  |3300  |[3000, 3300]            |3150.0            |0.6666666666666666|3000|3900|
|Jen    |Finance  |3900  |[3000, 3300, 3900]      |3400.0            |1.0               |3300|null|
|Kumar  |Marketing|2000  |[2000]                  |2000.0            |0.5               |null|3000|
|Jeff   |Marketing|3000  |[2000, 3000]            |2500.0            |1.0               |2000|null|
|James  |Sales    |3000  |[3000]                  |3000.0            |0.25              |null|4100|
|Robert |Sales    |4100  |[3000, 4100, 4100]      |3733.3333333333335|0.75              |3000|4100|


In [35]:
(
    df      
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('nth_value', F.nth_value(df.salary, 2).over(wspec))
        .withColumn('ntile', F.ntile(2).over(wspec))
        .withColumn('dense_rank', F.dense_rank().over(wspec))
        .withColumn('percent_rank', F.percent_rank().over(wspec))
        .withColumn('rank', F.rank().over(wspec))
).show(truncate=False)

+-------+---------+------+------------------------+---------+-----+----------+------------------+----+
|emp_id |dept_id  |salary|salaries_list           |nth_value|ntile|dense_rank|percent_rank      |rank|
+-------+---------+------+------------------------+---------+-----+----------+------------------+----+
|Maria  |Finance  |3000  |[3000]                  |null     |1    |1         |0.0               |1   |
|Scott  |Finance  |3300  |[3000, 3300]            |3300     |1    |2         |0.5               |2   |
|Jen    |Finance  |3900  |[3000, 3300, 3900]      |3300     |2    |3         |1.0               |3   |
|Kumar  |Marketing|2000  |[2000]                  |null     |1    |1         |0.0               |1   |
|Jeff   |Marketing|3000  |[2000, 3000]            |3000     |2    |2         |1.0               |2   |
|James  |Sales    |3000  |[3000]                  |null     |1    |1         |0.0               |1   |
|Robert |Sales    |4100  |[3000, 4100, 4100]      |4100     |1    |2     

### `UDF`

In [36]:
import pandas as pd

@F.pandas_udf('double', F.PandasUDFType.SCALAR)
def add_one(v):
    return v + 1

ratings_df.select(add_one(ratings_df.rating)).show()
ratings_df.withColumn(
    'plus_one', add_one(ratings_df.rating)
).show()

/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+---------------+
|add_one(rating)|
+---------------+
|            6.0|
|            4.0|
|            4.0|
|            5.0|
|            6.0|
|            4.0|
|            6.0|
|            6.0|
|            5.0|
|            5.0|
|            6.0|
|            5.0|
|            5.0|
|            5.0|
|            6.0|
|            5.0|
|            4.0|
|            5.0|
|            6.0|
|            5.0|
+---------------+
only showing top 20 rows



+-------+--------+------+---------+--------+
|user_id|movie_id|rating|timestamp|plus_one|
+-------+--------+------+---------+--------+
|      1|    1193|   5.0|978300760|     6.0|
|      1|     661|   3.0|978302109|     4.0|
|      1|     914|   3.0|978301968|     4.0|
|      1|    3408|   4.0|978300275|     5.0|
|      1|    2355|   5.0|978824291|     6.0|
|      1|    1197|   3.0|978302268|     4.0|
|      1|    1287|   5.0|978302039|     6.0|
|      1|    2804|   5.0|978300719|     6.0|
|      1|     594|   4.0|978302268|     5.0|
|      1|     919|   4.0|978301368|     5.0|
|      1|     595|   5.0|978824268|     6.0|
|      1|     938|   4.0|978301752|     5.0|
|      1|    2398|   4.0|978302281|     5.0|
|      1|    2918|   4.0|978302124|     5.0|
|      1|    1035|   5.0|978301753|     6.0|
|      1|    2791|   4.0|978302188|     5.0|
|      1|    2687|   3.0|978824268|     4.0|
|      1|    2018|   4.0|978301777|     5.0|
|      1|    3105|   5.0|978301713|     6.0|
|      1| 

In [37]:
@F.pandas_udf(ratings_df.schema, F.PandasUDFType.GROUPED_MAP)
# Input/output are both a pandas.DataFrame
def subtract_mean(pdf):
    return pdf.assign(rating=pdf.rating - pdf.rating.mean())

ratings_df.groupby('movie_id').apply(subtract_mean).show()

/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------+--------+------------+---------+
|user_id|movie_id|      rating|timestamp|
+-------+--------+------------+---------+
|    181|      31|   0.8865249|977087101|
|    195|      31|-0.113475084|991013952|
|    203|      31|   -2.113475|976929358|
|    223|      31|-0.113475084|976905652|
|    268|      31|   0.8865249|976647137|
|    368|      31|-0.113475084|976670975|
|    517|      31|   0.8865249|976204301|
|    524|      31|   -2.113475|976171096|
|    528|      31|   1.8865249|980039160|
|    531|      31|  -1.1134751|978973034|
|    536|      31|-0.113475084|976137228|
|    543|      31|   0.8865249|976159357|
|    616|      31|-0.113475084|975802599|
|    676|      31|   0.8865249|975684957|
|    678|      31|   0.8865249|989241973|
|    692|      31|-0.113475084|978375055|
|    699|      31|-0.113475084|975563262|
|    710|      31|-0.113475084|978586309|
|    752|      31|   -2.113475|975461295|
|    777|      31|-0.113475084|975520841|
+-------+--------+------------+---